In [ ]:
#https://www.kaggle.com/code/yzhou3/a-b-testing-in-r/notebook

In [ ]:
library(tidyverse)
list.files(path = "../input")
# read data
df<- read.csv("../input/ab-testing/ab_data.csv")
df <- data.frame(df)
colnames(df)
nrow(df)

#find out not aligned info between 'group' and 'landing_page'
notaligned_user=df %>% filter((df$group=='treatment'& landing_page == "old_page")|(df$group=='control'& landing_page == "new_page"))

#aligned info between 'group' and 'landing_page'
df1=df[(df$group=='control'& df$landing_page == "old_page")|(df$group=='treatment'& df$landing_page == "new_page"),]

In [ ]:
# find out unique id
unique_id <- unique(df1$user_id)
length(unique_id)
#if a user clicked several times, only keep the first result for analysis. 
df1 <- df1 %>% group_by(user_id) %>% arrange(timestamp)
df2=df1[!duplicated(df1$user_id),]            
nrow(df2)

In [ ]:
#H0:Pnew<=Pold  H1:Pnew>=Pold
library(lubridate)
df2$date<-as.Date(df2$timestamp)
df_new<-df2 %>% 
group_by(date,group) %>% 
summarize(conversion_rate = mean(converted))
head(df2)

In [ ]:
#ggplot
ggplot(df_new,aes(x=date,y=conversion_rate,
  color = group,group = group)) +
  geom_point(size = 2) +
  geom_line(lwd = 1) +
  labs(x = "date",  y = "Conversion Rate")

In [ ]:
#AB Testing: Organize variables and run logistic regression
df2$group = factor(df2$group, levels = c("control", "treatment"))
fit<-glm(converted ~ group, family = "binomial", data =df2) 
summary(fit)
# group treatment 0.19>0.05
#not significant, cannot reject H0
#think of new idea